# [LFU - Least Frequently Used Cache](https://leetcode.com/problems/lfu-cache/)

- `LFUCache(int capacity)`: Initializes the object with the capacity of the data structure.
- `int get(int key)`: Gets the value of the key if the key exists in the cache. Otherwise, returns -1.
- `void put(int key, int value)`:
    + Update the value of the key if present
    + Inserts the key if not already present
    + When the cache reaches its capacity
        + **invalidate** the least frequently used key
        + insert the new item
- Notes: when there is a tie (i.e., two or more keys with the same frequency), the **least recently used** key would be invalidated.

# Solution
- Notes: the important thing = manage and track `_least_freq`

<img src="assets/1.png" width="700"/>

#### Code

```C++
struct Node {
    int key;
    int val;

    Node(): key(0), val(0) {}
    Node(int k, int v): key(k), val(v) {}
};


class LFUCache {
private:
    unordered_map<int, int> _table_freq;
        // key -> freq
    unordered_map<int, list<Node>> _freq_queue;
        // freq -> list<Node>
    unordered_map<int, list<Node>::iterator> _table_node;
        // key -> Node itr

    const int DEFAULT_FREQ = 1;
    int _cap;
    int _least_freq;

public:
    LFUCache(int capacity): _cap(capacity), _least_freq(DEFAULT_FREQ) {
        _table_freq.clear();
        _freq_queue.clear();
        _table_node.clear();
    }

    void __update(int key, int value) {
        int old_freq = _table_freq[key];
        int new_freq = old_freq + 1;

        // Update _table_freq
        _table_freq[key] = new_freq;

        // Update _freq_queue and _least_freq
        _freq_queue[old_freq].erase(_table_node[key]);
        if(_freq_queue[old_freq].empty()) {
            _freq_queue.erase(old_freq);
            if(_least_freq == old_freq) _least_freq += 1;
        }
        _freq_queue[new_freq].push_front(Node(key, value));

        // Update _table_node
        _table_node[key] = _freq_queue[new_freq].begin();
    }
    void __add(int key, int value) {
        // Update _table_freq
        _table_freq[key] = DEFAULT_FREQ;

        // Update _freq_queue and _least_freq
        _freq_queue[DEFAULT_FREQ].push_front(Node(key, value));
        _least_freq = DEFAULT_FREQ;

        // Update _table_node
        _table_node[key] = _freq_queue[DEFAULT_FREQ].begin();
    }
    void __evict() {
        int key = _freq_queue[_least_freq].back().key;

        // Update _table_freq
        _table_freq.erase(key);

        // Update _freq_queue and _least_freq
        _freq_queue[_least_freq].pop_back();
        if(_freq_queue[_least_freq].empty()) _freq_queue.erase(_least_freq);
            // Notes: Dont need to update _least_freq
            //      After eviction --> __add(), update _least_freq = 1 there

        // Update _table_node
        _table_node.erase(key);
    }

    int get(int key) {
        if(_table_node.find(key) == _table_node.end()) return -1;

        int value = _table_node[key]->val;
        __update(key, value);
        return value;
    }

    void put(int key, int value) {
        if(_cap == 0) return;

        // Key in LFU -> Update new freq to key
        if(_table_node.find(key) != _table_node.end()) {
            __update(key, value);
        }

        // Key not in LFU, still have space -> Add new node with freq = 1
        else if(_table_freq.size() < _cap) {
            __add(key, value);
        }

        // Key not in LFU, reach capacity -> Evict then Add
        else {
            // Evict least freq and least recent node
            __evict();

            // Add new node with freq = 1
            __add(key, value);
        }
    }
};
```